## Modelo Supervisionado 

**Projeto**: Expansão por grupos — classificador de “Bom Desempenho” por região×grupo
**Objetivo**: Treinar e avaliar um RandomForest com pipeline (pré-processamento + OHE), escolher limiar ótimo por F1, salvar o modelo e ranquear candidatos.

### Configuração inicial, importações e parâmetros globais

- Importa bibliotecas essenciais para:
  - Manipulação e análise de dados (`pandas`, `numpy`)
  - Visualização (`matplotlib`)
  - Modelagem supervisionada (`scikit-learn`)
  - Persistência de modelos (`joblib`)

- Define configurações globais:
  - Supressão de warnings não críticos
  - Ajustes de exibição do pandas (linhas, colunas e largura)

- Estabelece parâmetros do projeto:
  - Caminhos para datasets e artefatos
  - Hiperparâmetros globais (ex.: `TOP_N`, `TOP_K`, `RANDOM_STATE`)

- Implementa função `read_csv_flex`:
  - Busca arquivos `.csv` em múltiplos diretórios possíveis
  - Testa diferentes separadores (`;`, `,`, autodetecção)
  - Retorna `DataFrame` válido ou lança erro com histórico de tentativas


In [ ]:
# ===== Imports principais =====
import glob
from pathlib import Path
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ===== Módulos do scikit-learn =====
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit  # validação estratificada
from sklearn.compose import ColumnTransformer                               # pré-processamento por tipo de coluna
from sklearn.pipeline import Pipeline                                       # pipeline unificado
from sklearn.impute import SimpleImputer                                    # imputação de valores faltantes
from sklearn.preprocessing import OneHotEncoder                             # codificação categórica
from sklearn.ensemble import RandomForestClassifier                         # classificador de árvores
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score,
                             average_precision_score, precision_recall_curve, roc_curve)  # métricas de avaliação
from sklearn.inspection import permutation_importance                       # interpretabilidade
from joblib import dump, load                                               # persistência de modelos

# ===== Configurações globais =====
warnings.filterwarnings("ignore", category=UserWarning)  # suprime avisos não críticos
pd.set_option("display.max_columns", None)              # exibe todas as colunas
pd.set_option("display.width", 220)                     # define largura máxima de exibição
pd.set_option("display.max_rows", 200)                  # aumenta limite de linhas exibidas

# ===== Parâmetros do projeto =====
CAMINHO_DF_HIST = "../../../database/dataset gerado/dataset_limpo.csv"                 # dataset histórico (limpo)
ARQ_MODELO      = Path("../../../database/dataset gerado/modelo_grupo_chilli_simplificado.joblib")    # arquivo para salvar/carregar modelo
ARQ_NOVAS       = "../../../database/dataset gerado/sugestoes_expansao_para_supervisionado.csv"  # sugestões do não supervisionado
TOP_N           = 10                                                 # número de categorias no ranking final
TOP_K           = 1                                                  # top-k localidades
FORCAR_TREINO   = False                                              # se True, força re-treinamento
LIMIAR          = None                                               # limiar de decisão (se não definido, será calibrado)
RANDOM_STATE    = 42                                                 # semente aleatória para reprodutibilidade

# ===== Função utilitária para leitura flexível de CSV =====
def read_csv_flex(base_path_or_stem: str) -> pd.DataFrame:
    """
    Lê um arquivo CSV de forma flexível, testando múltiplos caminhos e separadores.
    Retorna um DataFrame se encontrado, caso contrário levanta FileNotFoundError.
    """
    # Constrói lista de candidatos (caminho direto, com extensão, em pastas comuns)
    candidates = [
        Path(base_path_or_stem),
        Path(f"{base_path_or_stem}.csv"),
        Path("data") / f"{base_path_or_stem}.csv",
        Path("datasets") / f"{base_path_or_stem}.csv",
    ]
    # Acrescenta arquivos encontrados no diretório de forma recursiva
    for p in glob.glob(f"**/{Path(base_path_or_stem).stem}*.csv", recursive=True):
        candidates.append(Path(p))

    tried = []  # registra tentativas de leitura
    for path in candidates:
        if not path.exists():
            tried.append(str(path))
            continue
        # Testa diferentes separadores
        for sep in [None, ';', ',']:
            try:
                df = pd.read_csv(path, sep=sep, engine="python")
                print(f"✔ Arquivo carregado de: {path}")
                return df
            except Exception:
                tried.append(f"{path}(sep={sep})")
    # Se nenhum arquivo foi lido, lança erro com histórico de tentativas
    raise FileNotFoundError("CSV não encontrado. Tentativas: " + " | ".join(tried))


### Construção da base supervisionada (features + rótulo por REGIÃO × GRUPO)

- **Finalidade:** consolidar, em uma linha por par `REGIÃO × GRUPO`, as variáveis explicativas e o rótulo binário de desempenho.
- **Entradas principais:** `df_hist` com colunas de preço (bruto e líquido), desconto, identificador de loja, região da loja e grupo de produto.
- **Regras de negócio:**
  - O rótulo `Bom_Desempenho` marca como 1 os **Top-K grupos por região** (maior receita líquida por par `REGIÃO × GRUPO`); demais pares recebem 0.
  - As **features** incluem: receita total por região, número de lojas por região, receita por loja na região e **medianas por par `REGIÃO × GRUPO`** (preço de varejo e desconto).
- **Artefatos auxiliares:**
  - `catalogo` agrega metadados por grupo (ex.: `Grupo_Produto` e mediana de preço de varejo) para cruzar depois com as novas localidades.
  - `agg_reg` e `grp_meds` ficam disponíveis para análises e enriquecimentos adicionais.
- **Validações rápidas:**
  - Conferir presença das colunas exigidas antes de agrupar.
  - Conferir distribuição de `Bom_Desempenho` e a forma de `X_all`/`y_all`.
  - Visualizar distribuição do alvo com um gráfico de barras simples.


In [ ]:
# Carregar e preparar dados
df = pd.read_csv(CAMINHO_DF_HIST)

# Limpeza inicial dos dados
df = df.drop(columns=[
    "ID_Cliente","Dim_Cliente.Data_Nascimento","Dim_Cliente.Regiao_Cliente",
    "ID_Produto","Dim_Lojas.Nome_Emp","Dim_Lojas.Bairro_Emp","Dim_Lojas.Cidade_Emp",
    "Dim_Lojas.CANAL_VENDA","Dim_Lojas.Tipo_PDV","Dim_Lojas.Regiao"
], errors="ignore")

# Preparação dos dados para treino e teste
from sklearn.model_selection import train_test_split

# Definir variáveis preditoras (X) e alvo (y)
X = df.drop('Dim_Produtos.GRUPO_CHILLI', axis=1)
y = df['Dim_Produtos.GRUPO_CHILLI']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Imports adicionais para pré-processamento
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Identificar colunas categóricas e numéricas
categorical_columns = X.select_dtypes(include=['object']).columns
numeric_columns = X.select_dtypes(include=['int64', 'float64']).columns

# Compatibilidade entre versões do sklearn para o OneHotEncoder
if "sparse_output" in OneHotEncoder().get_params().keys():
    oh = OneHotEncoder(
        drop='first', 
        sparse_output=False,
        handle_unknown='ignore'  # Ignora categorias desconhecidas
    )  
else:
    oh = OneHotEncoder(
        drop='first', 
        sparse=False,
        handle_unknown='ignore'  # Ignora categorias desconhecidas
    )

# Criar preprocessador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_columns),
        ('cat', oh, categorical_columns)
    ])

# Criar pipeline com preprocessamento e modelo
dt_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(
        criterion='gini',
        max_depth=6,
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=42
    ))
])

# Treinar o modelo
dt_pipeline.fit(X_train, y_train)

# Fazer previsões
y_pred_dt = dt_pipeline.predict(X_test)

# Avaliar o desempenho
print("Acurácia:", accuracy_score(y_test, y_pred_dt))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred_dt))
print("\nMatriz de Confusão:\n", confusion_matrix(y_test, y_pred_dt))

In [ ]:
# Visualização 1: Gráfico de Barras das Métricas
import matplotlib.pyplot as plt
import numpy as np

metrics = classification_report(y_test, y_pred_dt, output_dict=True)
metrics_df = pd.DataFrame(metrics).transpose()

# Filter out the 'support' row and the macro/weighted averages
metrics_df = metrics_df.drop(['support', 'accuracy'], errors='ignore')
metrics_df = metrics_df.iloc[:-3]

metrics_df[['precision', 'recall', 'f1-score']].plot(kind='bar', figsize=(10, 6))
plt.title('Métricas de Classificação por Classe')
plt.ylabel('Valor')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Visualização 2: Heatmap da Matriz de Confusão
import seaborn as sns
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred_dt)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusão')
plt.xlabel('Valor Predito')
plt.ylabel('Valor Real')
plt.tight_layout()
plt.show()